## Channels
|**N**          |0  |1  |2  |3  |4  | 5 | 6 | 7 | 8 | 9 |
|---------------|---|---|---|---|---|---|---|---|---|---|
|**Sentinel-2A**|B02|B03|B04|B05|B06|B07|B08|B8A|B11|B12|
|**Standard**   |B  |G  |R  |RE1|RE2|RE3|N  |N2 |S1 |S2 |

## Classes:
|**N**        |1  |2  |3  |4  | 5 |
|-|-|-|-|-|-|
|**seg. mask**|open water|settlements|bair soil|forest|grassland|

## Indexes:
|short|long|type|formula|
|-|-|-|-|
LSWI|Land Surface Water Index|water|(N - S1)/(N + S1)

In [ ]:
INDEXES = {
    "BI": "((S1 + R) - (N + B))/((S1 + R) + (N + B))",
    "BNDVI": "(N - B)/(N + B)",
    "MGRVI": "(G ** 2.0 - R ** 2.0) / (G ** 2.0 + R ** 2.0)",
    "NDCI": "(RE1 - R)/(RE1 + R)",
    "NLI": "((N ** 2) - R)/((N ** 2) + R)",
}

In [ ]:
import torch.nn as nn
from DataVis import *
from LandcoverDataset import LandcoverDataset as LD
from Model import CNN, ResidualBlock
from utils import *
from random import choice as c

In [ ]:
BATCH = 4

# dl_train = LD(
#     img_path="datasets/USA/train/images/",
#     mask_path="datasets/USA/train/masks/",
#     batch_size=BATCH,
#     indexes=list(INDEXES.values()),
#     # n_random=10
# )
dl_valid = LD(
    img_path="datasets/USA/valid/images/",
    mask_path="datasets/USA/valid/masks/",
    batch_size=BATCH,
    transforms=False,
    indexes=list(INDEXES.values()),
    # n_random=10
)

In [ ]:
INPUT_CHANNELS = 10 + len(INDEXES)
N_CLASSES = 5
DROPOUT_RATE = 0.2

layers = [
    nn.Conv2d(INPUT_CHANNELS, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(32),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),

    ResidualBlock(64, 64),
    ResidualBlock(64, 64),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.MaxPool2d(kernel_size=2, stride=2),
    # nn.Dropout2d(p=DROPOUT_RATE),  # Consider removing dropout here

    ResidualBlock(128, 128),

    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.MaxPool2d(kernel_size=2, stride=2),

    ResidualBlock(256, 256),

    nn.Conv2d(256, 512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(512),

    ResidualBlock(512, 512),
    ResidualBlock(512, 512),

    nn.Conv2d(512, N_CLASSES, kernel_size=1),
    nn.Upsample(size=(512, 512), mode="nearest"),
]

model = CNN(layers)

In [ ]:
INPUT_CHANNELS = 10 + len(INDEXES)
N_CLASSES = 5

model_old = CNN([
    nn.Conv2d(INPUT_CHANNELS, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, N_CLASSES, kernel_size=1),
    nn.Upsample(size=(512, 512), mode='nearest')
])

In [ ]:
dl_valid.plot_sample(90)

In [ ]:
dl_valid.plot_sample(90,index="((N ** 2) - R)/((N ** 2) + R)")

In [ ]:
dl_valid.plot_sample(90,index="(G ** 2.0 - R ** 2.0) / (G ** 2.0 + R ** 2.0)")

In [ ]:
dl_test.plot_prediction(model_old, c(range(345)))

In [ ]:
model_old.train(dl_train, dl_valid, 3, 0.001, 0.45)

In [ ]:
model.pickle(f"models/res_blocks.torch")

In [ ]:
# model.unpickle("models/next.torch")
# model.unpickle("models/res_blocks.torch")
model_old.unpickle("models/deep_CNN_epoch=3,mm=0.46.torch")

In [ ]:
model_old.plot_training_history()

In [ ]:
dl_test = LD(
    img_path="datasets/Russia/test/images/",
    mask_path="datasets/Russia/test/masks/",
    batch_size=BATCH,
    transforms=False,
    indexes=list(INDEXES.values()),
    # n_random=10
)

In [ ]:
model_old.test(dl_valid)

In [ ]:
model_old.test(dl_test)

In [ ]:
model_old.plot_training_history()

In [ ]:
from random import choice as c

dl_valid.plot_prediction(model_old, c(range(200)))

In [ ]:
epochs_sum = 10
m_name = f"epochs={epochs_sum}"
model.pickle(f"models/{m_name}.torch")